In [1]:
import numpy as np 
import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
import glob

Using TensorFlow backend.


## Datashape

In [2]:
path_images = ['/Users/flavio/Downloads/Learning/Eric_images/database/SAXS/*.tif',
               '/Users/flavio/Downloads/Learning/Eric_images/database/WAXS/*.tif', 
               '/Users/flavio/Downloads/Learning/Eric_images/database/GIWAXS/*.tif',
               '/Users/flavio/Downloads/Learning/Eric_images/database/GISAXS/*.tif']

learnining_rate = 0.001
decay = 0.0001
epochs = 4
batch_size = 64

In [3]:
cont = 0
labels = []
database = []
for i in range(0,len(path_images)):
    image_names = glob.glob(path_images[cont])
    for name in image_names:
        im = io.imread(name)
        im = im.reshape((im.shape[0],im.shape[1],1))
        database.append(im)
        labels.append(cont)
    cont+=1
database = np.array(database,dtype=np.uint8)
labels = np.array(labels,dtype=np.int)
labels = np_utils.to_categorical(labels)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(database,labels, test_size= 0.2)

In [5]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))

model.add(Conv2D(filters=48, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))

model.add(Flatten())

model.add(Dense(192, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [6]:
model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 48)       

In [7]:
sgd = optimizers.SGD(lr=learnining_rate,
                    decay=decay)
model.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator()
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), 
                    steps_per_epoch=len(X_train)/batch_size, 
                    epochs=epochs)

Epoch 1/4
2/5 [===========>..................] - ETA: 41s - loss: 2.4074 - accuracy: 0.3516

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)

In [ ]:
result = model.predict_generator(datagen.flow(X_test))

In [ ]:
result = result.argmax(axis=1)

In [ ]:
print('Accuracy: ',accuracy_score(y_test.argmax(axis=1), result))

In [ ]:
model.save('lenet.h5')